In [71]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
pages_links = ["https://transcripts.foreverdreaming.org/viewforum.php?f=18",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=25",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=50",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=75",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=100",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=125",
               "https://transcripts.foreverdreaming.org/viewforum.php?f=18&start=150"]

In [108]:
#get all episode links in a page 
def get_all_links_in_page(url):
  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')
 
  urls = []
  for link in soup.find_all('a'):
    link = link.get('href')
    if "./viewtopic.php?" in link:
      urls.append("https://transcripts.foreverdreaming.org/" + link[2:])
  return urls[2:]

In [57]:
#get episodes transcript 
def episode_script(url):
  script = []
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, "html.parser")
  texts = soup.find_all('p')
  for text in texts:
    line = text.get_text()
    if ':' in line:
      script.append(line)
  return(script)

In [149]:
#turn transcript to df 
def script_to_df(script):
  script_list = []
  for line in script:
    res = re.split(':', line)
    if (res[0] != 'Editor') and (len(res) == 2):
      script_list.append(res)
  df = pd.DataFrame(script_list, columns = ['character', 'line'])
  return df

In [155]:
#now put it all together 
def main(pages_links):
  #get all links
  all_links = []
  for link in pages_links:
    links = get_all_links_in_page(link)
    all_links.append(links)
    flat_links = [item for sublist in all_links for item in sublist]
  full_script = []
  #get each episode script
  for link in flat_links[:-1]:
    script = episode_script(link)
    full_script.append(script)
  flat_script = [item for sublist in full_script for item in sublist]
    
  df = script_to_df(flat_script)

  return(df)

In [156]:
final_df = main(pages_links)

In [159]:
#write df to csv
final_df.to_csv('tvd_script.csv')